In [2]:
!pip install transformers peft bitsandbytes datasets trl accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 23.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# --- 1. Configuration ---

base_model_name = "microsoft/phi-3-mini-4k-instruct" # Base SLM to fine-tune
dataset_path = "/content/drive/MyDrive/train_ag_news.jsonl" # Path to your formatted training data

# --- !! CHOOSE WHERE TO SAVE THE ADAPTER !! ---
# Option A: Save to Google Drive (Recommended for persistence)
output_dir = "/content/drive/MyDrive/my-phi3-agnews-adapter"
print(f"Adapter will be saved to: {output_dir}")

Adapter will be saved to: /content/drive/MyDrive/my-phi3-agnews-adapter


In [5]:
# --- 2. Load Model, Tokenizer, and Quantization (QLoRA) ---
print("Loading base model and tokenizer (4-bit)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False # Important for training stability
model.config.pretraining_tp = 1 # Recommended setting

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # SFTTrainer generally expects right padding

Loading base model and tokenizer (4-bit)...


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [6]:
# --- 3. Configure LoRA ---
print("Configuring LoRA...")
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
# Note: get_peft_model is called internally by SFTTrainer if peft_config is provided

Configuring LoRA...


In [7]:
# --- 4. Load Your Formatted Data ---
print(f"Loading dataset from: {dataset_path}")
try:
    dataset = load_dataset("json", data_files=dataset_path, split="train")
    print(f"Dataset loaded successfully with {len(dataset)} examples.")
    # Optional: Inspect one example to ensure format is correct
    print("First example:\n", dataset[0]['text'])
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please ensure 'train_ag_news.jsonl' exists and is correctly formatted.")
    raise

Loading dataset from: /content/drive/MyDrive/train_ag_news.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded successfully with 1000 examples.
First example:
 <s>[INST] Classify the following news article into one of these categories: World, Sports, Business, or Sci/Tech. Article: 'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.' [/INST] World </s>


In [8]:
# --- 5. Set Up Training Arguments ---
print("Setting up training arguments...")
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,       # Adjust based on GPU memory (T4 usually handles 1 or 2)
    gradient_accumulation_steps=4,       # Effectively increases batch size to 2*4=8
    num_train_epochs=1,                  # Start with 1 epoch for faster iteration
    learning_rate=2e-4,
    logging_steps=25,                    # Log progress every 25 steps
    optim="paged_adamw_8bit",            # Memory-efficient optimizer
    save_strategy="steps",               # Save checkpoints periodically
    save_steps=100,                      # Save every 100 steps (adjust based on dataset size)
    # evaluation_strategy="no",            # We'll evaluate separately after training
    # push_to_hub=output_dir.startswith("YourUsername/"), # Set True if output_dir is HF repo
    fp16=False, # bf16 is generally better if available and QLoRA uses bfloat16 compute_dtype
    bf16=torch.cuda.is_bf16_supported(), # Use bf16 if supported by the GPU (T4 supports it)
    max_steps=-1,                        # Set max_steps instead of epochs if you prefer (e.g., 200)
    warmup_ratio=0.03,                   # Standard warmup
    group_by_length=True,                # Speeds up training slightly
    lr_scheduler_type="constant",        # Simple scheduler
    report_to="tensorboard",             # Optional: For visualizing training curves
    gradient_checkpointing=True,         # Saves memory
    gradient_checkpointing_kwargs={'use_reentrant':False}, # Recommended setting
)

Setting up training arguments...


In [9]:
# --- 6. Initialize the Trainer ---
print("Initializing SFTTrainer...")
trainer = SFTTrainer(
    model=model,                         # Pass the base model (PEFT applied internally)
    train_dataset=dataset,
    peft_config=peft_config,             # Pass LoRA config here
    # dataset_text_field="text",           # The column in your JSONL with the formatted string
    # max_seq_length=512,                  # Adjust based on your data and GPU memory
    # tokenizer=tokenizer,
    args=training_args,
    # packing=False,                       # Keep packing False for simpler instruction tuning setup
)

Initializing SFTTrainer...


Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
# --- 7. Start Fine-Tuning ---
print("\nStarting fine-tuning...")
train_result = trainer.train()
print("Fine-tuning finished.")


Starting fine-tuning...


Step,Training Loss
25,1.642300
50,1.391200
75,1.347200
100,1.357300
125,1.372400


Fine-tuning finished.


In [11]:
# --- 8. Save the Final Adapter ---
print(f"Saving the final adapter model to {output_dir}")
trainer.save_model(output_dir) # Saves the adapter config and weights

Saving the final adapter model to /content/drive/MyDrive/my-phi3-agnews-adapter


In [12]:
# Optional: Log metrics
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        1.0
  total_flos               =  2150898GF
  train_loss               =     1.4221
  train_runtime            = 0:24:24.67
  train_samples_per_second =      0.683
  train_steps_per_second   =      0.085
